# Dependencies


In [22]:

import tweepy
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
!pip freeze | grep tweepy

tweepy==3.7.0


# Twitter Api Key

In [23]:

consumer_key = 'wxgWPzoSV1jJ4w9ickrJ7vEbi'
consumer_secret = 'bXX0bMVXyeff3BtlchzBiNyoSxAWNaUfC7r2TlYjKygeK2o6XF'
access_token = '1041110068735832064-ZyCKSxnFVqjbRBf7yQnQQ1ES4LHWoO'
access_token_secret = 'Iap9tp6knHDHkxVhAolscYHQPir0MvPBJyN4lnzm8k2mF'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)  # twitter consumer key and consumer secret
auth.set_access_token(access_token, access_token_secret)  # twitter access token and access_token_secret

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


# Creating wordcount and Use of For loops¶

# Python Function Parameters

Creating a pair of braces with an empty dictionary



The param() method creates a serialized representation of an array or an object.

The input function has a return variable.

len() function is an inbuilt function in Python programming language that returns the length of the string.

The split() method splits a string into a list.

Sorted() function has an optional parameter called 'key' which takes a function as its value. 

This key function transforms each element before sorting, it takes the value and returns 1 value which is then used within sort instead of the original value.



In [3]:

def top_word_count(text, min_word_length=2, top=50):
    """
    Get word count out of text
    It also removes all single character words
    :param text: input text
    :param min_word_length: min characters to consider as a word
    :param top: top number of words to be returned
    :return: top words
    """
# creat a pair of braces with an empty dictionary: {}.    
    wordcount = {}
    for word in text.split():
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1
    final_wordcount = {}
    for key, value in wordcount.items():
        if (len(key) >= min_word_length):
            final_wordcount[key] = value
    final_wordcount = sorted(final_wordcount.items(), key=lambda x: x[1], reverse=True)
    return final_wordcount[:top]


# Summarize tweeter groups

In [4]:
def get_group_summary(group, category):
    """
    Summarize tweeter groups
    :param group:
    :return: summary json
    """
    group['Text'] = group['Text'].str.replace(r'[{}]+'.format(category), '')
    top_words = top_word_count(" ".join(group['Text'].values))
    return {'word_count': top_words,
            'retweet_sum': group['Retweet Count'].sum(),
            'favorite_sum': group['Favorite Count'].sum(),
            'reply_count': group[group['Is Reply'] == True]['Is Reply'].count(),
            'quote_count': group[group['Quote'] == True]['Quote'].count()
            }

# param categories: categories which you would like to compare


# param limit: limit is the number of tweets to get per category with hash or no hash
    
   

In [5]:
def get_tweets(categories, limit):
    """
    :param categories: categories which you would like to compare
    :param limit: limit is the number of tweets to get per category with hash or no hash
    :return:
    """
    new_columns = ['category', 'Is Hashtag', 'Text', 'Quote', 'Retweet Count', 'Favorite Count', 'Is reply']
    data = list()
    for category in categories:

        tweets = api.search(category, lang='en', count=limit)
        
        for tweet in tweets["statuses"]: # iterate tweets without hash
            # add the details for each tweet without hash in this category
            text = tweet["text"]
            quote = tweet["is_quote_status"]
            retweet_count = tweet["retweet_count"]
            favorite_count = tweet["favorite_count"]
            is_reply = False if tweet["in_reply_to_status_id"] == None else True
            row = (category, False, text, quote, retweet_count, favorite_count, is_reply)
            data.append(row)
            
        tweets = api.search(f"#{category}", lang='en', count=limit)
        # Using # instead of @
        for tweet in tweets["statuses"]: # iterate tweets with hashtag
            # add the details for each tweet with hash in this category
            text = tweet["text"]
            quote = tweet["is_quote_status"]
            retweet_count = tweet["retweet_count"]
            favorite_count = tweet["favorite_count"]
            is_reply = False if tweet["in_reply_to_status_id"] == None else True
            row = (category, True, text, quote, retweet_count, favorite_count, is_reply)
            data.append(row)
        print(f'processed {category}') # show progress medf.to_csv("sample_2.csv", quoting=csv.QUOTE_ALL)

    df = pd.DataFrame(data, columns=new_columns)
    return df

# Cleaning Data

convert all text to lower case

Remove all special characters except text and white space characters.

Remove rt(retweet) and category as it will be contained in many tweets

In [24]:
def compare_tweets_by_categories(df):
    """
    :param df: tweets dataframe
    :return: list of dictionaries of categories and hashvalues, along with summary
    """
    # convert all text to lower case
    # Remove all special characters except text and white space characters
    df['Text'] = df['Text'].str.lower().str.replace('[^\w\s]', '')
    # Remove rt(retweet) and category as it will be contained in many tweets
    df['Text'] = df['Text'].str.replace(r'[_|rt]+', '')

    result = df.groupby(['category', 'Is Hashtag'])
    result_list = list()
    for name, group in result:
        result_list.append(
            {
                "category": name[0],
                "is_hashtag": name[1],
                "summary": get_group_summary(group, name[0])
            }
        )
    return result_list


# Collecting data from Twitter using API
entertainment, sports, art and design, government, politics
    # health
    # research
collecting categories in a dataframe

 calls the method, takes two params (list of categories and number of tweets to get per category

In [25]:
  
categories = ['health','entertainment','sports','arts and design','government','politics','research']

# collect http://localhost:8888/notebooks/Twitter_Trends_Project3/twitter_charts%20(7).ipynb#tweets with categories and append to sample_1.csv
df = get_tweets(categories, 500)



processed health
processed entertainment
processed sports
processed arts and design
processed government
processed politics
processed research


# Using plotly for basic-bar

In [26]:

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [27]:
df.head(200)

,category,Is Hashtag,Text,Quote,Retweet Count,Favorite Count,Is reply
0,health,False,@FranzWithFrenz 24 - health,False,0,0,True
1,health,False,@David_E_Squires @AOC I just threw down $500 a...,False,0,0,True
2,health,False,"RT @minnieyjm: fansites do ads, help promote t...",False,112,0,False
3,health,False,RT @amyklobuchar: I love my dad. He struggled ...,False,158,0,False
4,health,False,RT @ClearityFnd: Two key points in this articl...,False,2,0,False
5,health,False,RT @icrowdfundbuzz: Check out an #investment i...,False,1,0,False
6,health,False,RT @LungAssociation: We strongly support the b...,False,7,0,False
7,health,False,RT @JamilSmith: All abortion bans punish women...,False,1441,0,False
8,health,False,RT @GOPChairwoman: The #BeBest initiative is g...,False,1538,0,False
9,health,False,The Health Benefits of Slippery Elm Tea https:...,False,0,0,False


# ploting plotly

In [28]:

import plotly.plotly as py
import plotly.graph_objs as go


df_category = pd.DataFrame(df['category'])
df_retweet = pd.DataFrame(df['Retweet Count'])
data = [go.Bar(
            x= df['category'],
            y=df['Retweet Count']
    )]
#ploting plotly
py.iplot(data, filename='basic-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~SairaJahangir/0 or inside your plot.ly account where it is named 'basic-bar'


# Using plotly to compare category/Retweet Count and category/Favorite Count
 This comparison gives us a suggestion that Retweet has a much higher tracing when we compare with Favorite
 trace1 comparing category with Retweet Count

In [29]:
import plotly.plotly as py
import plotly.graph_objs as go


trace1 = go.Bar(
    x= df['category'],
    y=df['Retweet Count']
   
)

# trace 2 comapring Category with Favorite Count
trace2 = go.Bar(
    x=['category'],
    y=['Favorite Count'],
   
)
#Layout trace 1 and trace 2 into one gro
data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~SairaJahangir/0 or inside your plot.ly account where it is named 'basic-bar'


# Here getting the list of category and favorite count
 x is array of category
 
 y is array of Favorite Count and Retweet Count 

In [30]:

data

[Bar({
     'x': array(['health', 'health', 'health', ..., 'research', 'research', 'research'],
                dtype=object),
     'y': array([  0,   0, 112, ...,   6,   0,   4])
 }), Bar({
     'x': ['category'], 'y': ['Favorite Count']
 })]

# Machine learning and NLTK 

NLTK is a leading platform for building Python programs to work with human language dat

The nltk.corpus package defines a collection of corpus reader classes, which can be used to access the contents of a diverse set of corpora

The NLTK corpus is a massive dump of all kinds of natural language data sets

When the nltk.corpus module is imported, it automatically creates a set of corpus reader instances that can be used to access the corpora in the NLTK data distribution.

Corpus is the equivalent of “dataset” in a general machine learning task.

Removing stop words with NLTK in Python

# Stop Words:
Stop Words: A stop word is a commonly used word (such as “the”, “a”, “an”, “in”) that a search engine has been programmed to ignore.

NLTK provides a function called word_tokenize() for splitting strings into tokens (nominally words). It splits tokens based on white space and punctuation.       

CLEANING TEXT FOR NATURAL LANGUAGE PROCESSING TASKS IN MACHINE LEARNING IN PYTHON:Tokenization is the first step in text analytics.

The process of breaking down a text paragraph into smaller chunks such as words or sentence is called Tokenization.

http://www.nltk.org/api/nltk.tokenize.html(Take a look at the other tokenizing options that nltk provides)

construct a dictionary with the built-in dict() function.

In [31]:

import nltk
from nltk.corpus import stopwords

def addWords(text,categoryDict):
# import the text file and tokenize it

   
    unwantedSymbols = ",.#()-/@"
    for symbol in unwantedSymbols:
        text = text.replace(symbol,"")
        
    tokens = [t for t in text.lower().split()]
# Frequency Distribution
    freq = nltk.FreqDist(tokens)
 
    for key,val in freq.items():
        if not key in stopwords.words('english'):
            categoryDict[key] = categoryDict[key] + val


# Collections in Python
Collections in Python are containers that are used to store collections of data, for example, list, dict, set, tuple etc

A defaultdict works exactly like a normal dict, but it is initialized with a function (“default factory”) that takes no arguments and provides the default value for a nonexistent key.

In Python, anonymous function means that a function is without a name. As we already know that def keyword is used to define the normal functions and the lambda keyword is used to create anonymous functions.

https://docs.python.org/3.5/tutorial/controlflow.html#lambda-expressions

In order to make a little easier, I created a small helper function to create a dataframe showing all the unique values.

A better way to loop through rows, if loop you must, is with the iterrows() method. iterrows() is a generator that iterates over the rows of the dataframe and returns the index of each row, in addition to an object containing the row itself.

Here creating dictionary containing each category and iterating over the category 

In [32]:


from collections import defaultdict
wordsPerCategory = {category : defaultdict(lambda : 0) for category in list(df.category.unique())}
# here data frame counting words for each category 
# each row extract text and receives the text and adds the counting of the words

for i, row in df.iterrows():
    addWords(row.Text,wordsPerCategory[row.category])
    

In [33]:
wordsPerCategory

{'health': defaultdict(<function __main__.<dictcomp>.<lambda>()>,
             {'franzwithfrenz': 1,
              '24': 1,
              'health': 121,
              'david_e_squires': 1,
              'aoc': 1,
              'threw': 1,
              '$500': 1,
              'ob': 1,
              'today': 4,
              'many': 4,
              'women': 4,
              'country': 1,
              'can’t': 1,
              'afford': 2,
              'that?…': 1,
              'https:tcottqg0zspc3': 1,
              'rt': 76,
              'minnieyjm:': 1,
              'fansites': 1,
              'ads': 1,
              'help': 7,
              'promote': 1,
              'boys': 3,
              'sell': 1,
              'fanmade': 1,
              'goods': 1,
              'used': 2,
              'profit': 1,
              'buy': 2,
              'present': 1,
              'supple…': 1,
              'amyklobuchar:': 1,
              'love': 2,
              'dad': 1,
        

# Sorting the data 

Sorted() function has an optional parameter called 'key' which takes a function as its value. This key function transforms each element before sorting, it takes the value and returns 1 value which is then used within sort instead of the original value.

To loop every key and value from a dictionary – for k, v in dict.items ():

In [34]:
# sorted() function has an optional parameter called 'key' which takes a function as its value. This key function transforms each element before sorting, it takes the value and returns 1 value which is then used within sort instead of the original value.
# To loop every key and value from a dictionary – for k, v in dict.items():
for k,v in wordsPerCategory.items():
    print(k)
    for value in sorted(v.items(), key =lambda x : -x[1])[:10]:
        print("\t",value[0],value[1])

health
	 health 121
	 rt 76
	 mental 21
	 happy 20
	 life 19
	 new 17
	 go 8
	 &amp; 8
	 i’m 8
	 help 7
entertainment
	 rt 72
	 entertainment 68
	 3mxnny_12: 19
	 lunch 19
	 https:tcolkpm5h1cfk 19
	 join 10
	 fun 9
	 concerns 9
	 next 9
	 move 9
sports
	 sports 87
	 rt 81
	 win 12
	 tyra 11
	 banks 11
	 follow 11
	 would 10
	 like 10
	 get 10
	 first 9
arts and design
	 rt 82
	 design 53
	 arts 47
	 met 22
	 rich 22
	 frustrating" 22
	 vanessa 22
	 friedman 22
	 roberta 22
	 smith 22
government
	 rt 73
	 government 64
	 👏🏼 15
	 us 13
	 house 9
	 got 9
	 info 9
	 charliekirk11: 8
	 reward: 8
	 call 8
politics
	 rt 92
	 politics 42
	 trump 17
	 house 16
	 us 13
	 would 10
	 new 9
	 like 9
	 &amp; 9
	 news 8
research
	 rt 107
	 research 105
	 new 21
	 said 13
	 development 13
	 report 13
	 ~ 13
	 killdbykill: 12
	 pharrell 12
	 refer 12


# sklearn: Simple and efficient tools for data mining and data analysis

bag of words vectors

ngram vectorized converts text into a vector of words frequency

how are you -> 0,0,...,1,...,1,...,1,....0,0

# CountVectorizer: count words and creats a vector

vector is just the frequency of each wordsfor example:describes which words are is a text, we have a list of words such as

the, a , word , unicorn... and this will help count the words as 2,0,1,0,... and so.

Using raw text, it might get better results, preprocessing it

For example converting everything to lowercase and removing stopwords

Here we know the X which is input and Y which we want to predict and that is category

# N-grams are simply all combinations of adjacent words or letters of length n that you can find in your source text.

For example, given the word fox, all 2-grams (or “bigrams”) are fo and ox. You may also count the word boundary – that would expand the list of 2-grams to #f, fo, ox, and x#, where # denotes a word boundary.

# The basic point of n-grams is that they capture the language structure from the statistical point of view.




In [35]:

from sklearn.feature_extraction.text import CountVectorizer

ngram_vectorizer = CountVectorizer()

ngram_vectorizer.fit(df['Text'])
X = ngram_vectorizer.transform(df['Text'])
Y = df.category.astype('category')

In [36]:
Y.head()

0    health
1    health
2    health
3    health
4    health
Name: category, dtype: category
Categories (7, object): [arts and design, entertainment, government, health, politics, research, sports]

# Training and Testing data 
Use train_test_split to create training and testing data
80% Of the model we use to train and 20% we use to test the model


In [37]:
# Use train_test_split to create training and testing data
# 80% Of the model we use to train and 20% we use to test the model

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,train_size=0.8)

/Users/saira/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



# Create a Linear Regression Model

Fit (train) or model using the training data

In [38]:

from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(X_train,y_train.cat.codes)
lr.score(X_test,y_test.cat.codes)

0.6765017509863853

# Random Forest Classifer

It's better to use a classifier for this task since there is a limited number of options to predict which are clearly a set of classes to predict.

Create a Random Forest Classifier.

The reason of using Random Forest Classifier as the Linear Regression model score didn't worked well.

In [39]:

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=50)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.7533039647577092